In [1]:
%pip install --upgrade --quiet wikipedia


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /usr/local/Cellar/jupyterlab/4.2.3/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_community.tools import DuckDuckGoSearchResults

In [5]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
search = DuckDuckGoSearchResults()

In [6]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [7]:
from langchain_core.prompts import ChatPromptTemplate

In [19]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant.  For answering the user query, look for infomration using DuckDuckGo and Wikipedia and then give the final answer"),
        ("placeholder", "{history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}")
    ]
)

In [10]:
tools = [search, wikipedia]

In [11]:
from langchain_openai import ChatOpenAI

In [12]:
llm = ChatOpenAI()

In [13]:
from langchain.agents import create_tool_calling_agent

In [14]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [15]:
from langchain.agents import AgentExecutor

In [16]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [20]:
agent_executor.invoke({"input": "What is the latest news in the U.S. 2024 election"})



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_results_json` with `{'query': 'latest news U.S. 2024 election'}`


[snippet: Aug. 23. 47% Trump Trump. 49% Harris Harris. Ruth Igielnik Staff editor, polling. The current polling averages capture where the race stands as Democrats wrap up their convention in Chicago ..., title: Election 2024 Polls: Harris vs. Trump | The New York Times, link: https://www.nytimes.com/interactive/2024/us/elections/polls-president.html], [snippet: An unsettled and unpredictable 2024 presidential race is entering its final stretch, with Vice President Kamala Harris looking to build on the momentum of this week's Democratic National ..., title: With the conventions over, Harris and Trump begin the final sprint to ..., link: https://edition.cnn.com/2024/08/24/politics/harris-trump-2024-race/index.html], [snippet: Robert F. Kennedy Jr. is suspending his independent campaign for the presidency, he said Friday at an event in Phoenix, the latest shif

{'input': 'What is the latest news in the U.S. 2024 election',
 'output': 'The latest news in the U.S. 2024 election includes updates on the polling averages between Vice President Kamala Harris and former President Donald Trump, the final sprint of the presidential race, Robert F. Kennedy Jr. suspending his independent campaign for the presidency, and ongoing coverage by news outlets like The New York Times and BBC News. You can find more details on these topics by visiting the respective links:\n1. [Election 2024 Polls: Harris vs. Trump | The New York Times](https://www.nytimes.com/interactive/2024/us/elections/polls-president.html)\n2. [With the conventions over, Harris and Trump begin the final sprint to ... | CNN](https://edition.cnn.com/2024/08/24/politics/harris-trump-2024-race/index.html)\n3. [Election 2024 Live Updates: R.F.K. Jr. Suspends Independent Bid for ... | The New York Times](https://www.nytimes.com/live/2024/08/23/us/harris-dnc-trump-election)\n4. [US Election 2024 |

## Example with memory

In [21]:
store = {}

In [25]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [24]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [27]:
agent_with_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    import_messages_key="input",
    history_messages_key="history"
)

In [30]:
agent_with_history.invoke(
    {"input": "What is the latest news in the U.S. 2024 election"},
    config={"configurable": {"session_id": "sess1"}}    
)



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_results_json` with `{'query': 'latest news U.S. 2024 election'}`


[snippet: The latest New York Times/Siena College polls find an essentially tied race across the four key Sun Belt battleground states. A dead heat in these states is not great news for Trump, who may need ..., title: Election 2024 Polls: Harris vs. Trump | The New York Times, link: https://www.nytimes.com/interactive/2024/us/elections/polls-president.html], [snippet: An unsettled and unpredictable 2024 presidential race is entering its final stretch, with Vice President Kamala Harris looking to build on the momentum of this week's Democratic National ..., title: With the conventions over, Harris and Trump begin the final sprint to ..., link: https://edition.cnn.com/2024/08/24/politics/harris-trump-2024-race/index.html], [snippet: Robert F. Kennedy Jr. is suspending his independent campaign for the presidency, he said Friday at an event in Phoenix, the lates

{'input': 'What is the latest news in the U.S. 2024 election',
 'history': [],
 'output': 'The latest news in the U.S. 2024 election includes:\n\n1. The latest New York Times/Siena College polls show an essentially tied race across the four key Sun Belt battleground states. For more information, you can visit [The New York Times article](https://www.nytimes.com/interactive/2024/us/elections/polls-president.html).\n\n2. The 2024 presidential race is described as unsettled and unpredictable as Vice President Kamala Harris aims to build on the momentum from the Democratic National Convention. Read more on this topic at [CNN](https://edition.cnn.com/2024/08/24/politics/harris-trump-2024-race/index.html).\n\n3. Robert F. Kennedy Jr. has announced the suspension of his independent campaign for the presidency, adding to the fast-evolving 2024 campaign landscape. You can find updates on this development at [The New York Times](https://www.nytimes.com/live/2024/08/23/us/harris-dnc-trump-electio

In [31]:
agent_with_history.invoke(
    {"input": "What were the battleground states referenced by the poll"},
    config={"configurable": {"session_id": "sess1"}}    
)



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'battleground states in 2020 US election'}`


Page: 2020 United States presidential election
Summary: The 2020 United States presidential election was the 59th quadrennial presidential election, held on Tuesday, November 3, 2020. The Democratic ticket of former vice president Joe Biden and the junior U.S. senator from California Kamala Harris defeated the incumbent Republican president, Donald Trump, and vice president, Mike Pence. The election took place against the backdrop of the global COVID-19 pandemic and related recession. The election saw the highest voter turnout by percentage since 1900, with each of the two main tickets receiving more than 74 million votes, surpassing Barack Obama's record of 69.5 million votes from 2008. Biden received more than 81 million votes, the most votes ever cast for a candidate in a U.S. presidential election.
In a competitive primary that featured the most candidates fo

{'input': 'What were the battleground states referenced by the poll',
 'history': [HumanMessage(content='What is the latest news in the U.S. 2024 election'),
  AIMessage(content='The latest news in the U.S. 2024 election includes:\n\n1. The latest New York Times/Siena College polls show an essentially tied race across the four key Sun Belt battleground states. For more information, you can visit [The New York Times article](https://www.nytimes.com/interactive/2024/us/elections/polls-president.html).\n\n2. The 2024 presidential race is described as unsettled and unpredictable as Vice President Kamala Harris aims to build on the momentum from the Democratic National Convention. Read more on this topic at [CNN](https://edition.cnn.com/2024/08/24/politics/harris-trump-2024-race/index.html).\n\n3. Robert F. Kennedy Jr. has announced the suspension of his independent campaign for the presidency, adding to the fast-evolving 2024 campaign landscape. You can find updates on this development at 